# 1. Introduction

This notebook connects to the Fresh Produce Database on SQL, cleans the tables and save the cleaned tables in the database.

!pip install pyodbc 

In [2]:
#!pip install sqlalchemy
#!conda install pymssql
#apt-get -qq install python-pymssql python3-pymssql

In [36]:
#Importing modules
import pandas as pd
import numpy as np
import pyodbc

#visualisation
from dataprep.eda import *
import matplotlib.pyplot as plt
import seaborn as sns

# SQL connectors 
from sqlalchemy import create_engine,event
from sqlalchemy.engine.reflection import Inspector
from sqlalchemy.engine import reflection

import urllib

import os              # For operating system related activities
from tqdm import tqdm  # For monitroing the progress

## Local Connection SQL

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-TTAV4PQ\SQLEXPRESS;'
                      'Database=Fresh_Produce_Market_Data;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

cpt_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Capetown_Fresh_produce_market',conn)
dbn_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Durban_Fresh_produce_market',conn)
jhb_comm_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_commodity_raw',conn)
jhb_con_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_container_raw',conn)
jhb_comb_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_product_combination_raw',conn)
woolworths_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.woolworths_Prices',conn)
pnp_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.PickNPay_Prices',conn)
shoprite_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Shoprite_Prices',conn)
#print(sql_query)
#print(type(sql_query))

## Remote Connection (RDS)

In [1]:
# creating a connection to the remote database (RDS)
cnxn = pyodbc.connect(
    DRIVER='{SQL Server}',
    SERVER='',
    DATABASE='',
    uid='',
    pwd='') 

NameError: name 'pyodbc' is not defined

In [78]:
# For sql queries 
cur = cnxn.cursor()

In [61]:
#Creating the pandas dataframe to store the data imported from RDS instance:
cpt_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Capetown_Fresh_produce_market',cnxn)
dbn_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Durban_Fresh_produce_market',cnxn)
jhb_comm_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_commodity_raw',cnxn)
jhb_con_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_container_raw',cnxn)
jhb_comb_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_product_combination_raw',cnxn)
woolworths_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.woolworths_Prices',cnxn)
pnp_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.PickNPay_Prices',cnxn)
shoprite_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Shoprite_Prices',cnxn)

In [7]:
# Exploring the data
cpt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5771 entries, 0 to 5770
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   item             5771 non-null   object
 1   description      5771 non-null   object
 2   grade            5771 non-null   object
 3   container        5771 non-null   object
 4   container_count  5771 non-null   int64 
 5   mass             5771 non-null   object
 6   low_price        5771 non-null   object
 7   high_price       5771 non-null   object
 8   avg_price        5771 non-null   object
 9   date_updated     5771 non-null   object
dtypes: int64(1), object(9)
memory usage: 451.0+ KB


In [24]:
cpt_df.head()

,item,description,grade,container,container_count,mass,low_price,high_price,avg_price,date_updated
0,MUBR,Mushroom Brown,,Prepack Punnet (5kg),0,5.00,600.00,600.00,600.00,25/08/2020 11:44 AM
1,MUOY,Mushroom Oyster,,Prepack Punnet (10kg),0,10.00,"1,000.00","1,000.00","1,000.00",25/08/2020 11:44 AM
2,MUWH,Mushroom White,,Prepack Punnet (5kg),0,5.00,250.00,250.00,250.00,25/08/2020 11:44 AM
3,NEDB,NECTARINES DIAMOND BRIGHT,1M,10*500g Punnet,0,5.00,250.00,250.00,250.00,25/08/2020 11:44 AM
4,NEOT,NECTARINES OTHER,2M,10*500g Punnet,0,5.00,220.00,220.00,220.00,25/08/2020 11:44 AM


In [6]:
cpt_df_cleaned.head()

,index,item,description,grade,container,container_count,mass,low_price,high_price,avg_price,date_updated
0,0,APCR,APPLE CRIPPS RED,1M,ECONOPACK (12kg),150,12.0,110.0,110.0,110.00,2020-08-21 11:36:00
1,1,APER,APPLE EARLY RED ONE,1L,MARK 4 (18.3kg),100,18.3,120.0,120.0,120.00,2020-08-21 11:36:00
2,2,APER,APPLE EARLY RED ONE,1M,MARK 4 (18.3kg),135,18.3,120.0,140.0,121.23,2020-08-21 11:36:00
3,3,APER,APPLE EARLY RED ONE,1X,MARK 4 (18.3kg),90,18.3,120.0,120.0,120.00,2020-08-21 11:36:00
4,4,APGA,APPLE GALA,1L,MARK 4 (18.3kg),110,18.3,160.0,160.0,160.00,2020-08-21 11:36:00


In [ ]:
cpt_df_cleaned

In [62]:
#jhb_comm_df.head()
#jhb_con_df.head()
jhb_comb_df.head()

,rowid,date,commodity,container,unit_mass,product_combination,total_value_sold,total_qty_sold,total_kg_sold,average,highest_price,ave_per_kg,highest_price_per_kg
0,1,20 August 2020,AMADUMBE,20KG POCKET,20,"*,*,L,*,*",R0.00,0,0.0,R0.00,R0.00,R0.00,R0.00
1,2,20 August 2020,APPLES,10 X 1KG ECONO PACK CARTON,10,"GOLDEN DELICIOUS,CL 1,*,*,*",R100.00,1,10.0,R100.00,R100.00,R10.00,R10.00
2,3,20 August 2020,APPLES,10 X 1KG ECONO PACK CARTON,10,"ROYAL GALA,CL 1,*,*,*",R0.00,0,0.0,R0.00,R0.00,R0.00,R0.00
3,4,20 August 2020,APPLES,11KG JUMBLE CARTON,11,"GOLDEN DELICIOUS,CL 2,L,*,*",R0.00,0,0.0,R0.00,R0.00,R0.00,R0.00
4,5,20 August 2020,APPLES,11KG JUMBLE CARTON,11,"TOPRED,CL 2,L,*,*",R0.00,0,0.0,R0.00,R0.00,R0.00,R0.00


# Creating Functions to clean the data

## The function to clean Rand Value columns  and normalise them

In [63]:
def rand_value(df, column):
    """
    Takes a dataframe and a column with rand value to be cleaned as arguments, 
    returns a dataframe, with two new columns with rand value for one day and
    Month To Date rand value. The initial column that was added as argument is removed from the dataframe.
    """ 
    
    df["MTD_total_value_sold_(R)"]=np.nan
    df["total_value_sold_(R)"]=np.nan
    for i in range(len(df[column])):
        df["MTD_total_value_sold_(R)"][i]=round(float(df[column][i].split("R")[-1].replace(',','')),2)
        df["total_value_sold_(R)"][i]=round(float(df[column][i].split("R")[1].split("M")[0].replace(",","")),2)
    
    df1=df.copy()    
    df1.drop(column,axis=1, inplace=True)
    return df1
    

## Cleaning the Johanneburg market Rand Values 
* Using the function

In [31]:
# cleaning the rand value column of commodity table and storing it in a new dataframe
jhb_comm_df_cleaned=rand_value(jhb_comm_df,"total_value_sold")
# cleaning the rand value column of the container dataframe and storing
jhb_con_df_cleaned=rand_value(jhb_con_df,"value_sold")

C:\Users\pitso\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\pitso\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [32]:
#Visualising the new dataframes
jhb_comm_df_cleaned.head()
jhb_con_df_cleaned.head()

,rowid,date,commodity,container,qty_available,qty_sold,kg_sold,average_price_per_kg,MTD_total_value_sold_(R),total_value_sold_(R)
0,1,20 August 2020,AMADUMBE,20KG POCKET,2,0MTD: 97,"0MTD: 1,940",R0,39870.0,0.0
1,2,20 August 2020,APPLES,10 X 1KG ECONO PACK CARTON,17,1MTD: 547,"10MTD: 5,470",R10,41932.0,100.0
2,3,20 August 2020,APPLES,11KG JUMBLE CARTON,343,"17MTD: 3,170","187MTD: 34,870",R6.36,218914.0,1190.0
3,4,20 August 2020,APPLES,12 X 1KG ECONO PACK CARTON,3233,"432MTD: 17,353","5184MTD: 208,236",R7.86,1454572.0,40738.0
4,5,20 August 2020,APPLES,12.5KG M6 CARTON,12,0MTD: 286,"0MTD: 3,575",R0,16903.0,0.0


## The function to clean quantities and weight columns simultaneuosly

In [64]:
def sold_qty_kg(df,quatity_sold, weight_sold):
    """
    Takes a dataframe and a two column with quantity sold and weight sold to be cleaned as arguments. 
    Returns a dataframe, with four new columns with measurements for one day and month to date measurements,
    of quantity sold and weight sold for one day and Month To Date. The initial columns that were added as argument 
    are removed from the new dataframe that is returned.
    """ 
    
    df["Total_quatity_sold"]=np.nan
    df["MTD_Total_quatity_sold"]=np.nan
    df["Total_kg_sold"]=np.nan
    df["MTD_total_kg_sold"]=np.nan
    for i in range(len(df[quatity_sold])):
        df["Total_quatity_sold"][i]=int(df[quatity_sold][i].split("M")[0].replace(",",""))
        df["MTD_Total_quatity_sold"][i]=int(df[quatity_sold][i].split(":")[-1].replace(',',''))
        df["Total_kg_sold"][i]=float(df[weight_sold][i].split("M")[0].replace(",",""))
        df["MTD_total_kg_sold"][i]=float(df[weight_sold][i].split(":")[-1].replace(',',''))
    df1=df.copy()
    df1.drop([quatity_sold,weight_sold],axis=1, inplace=True)
    return df1

## Cleaning the Joburg Market taables' quantity sold and weight sold column

In [34]:
jhb_comm_df_cleaned= sold_qty_kg(jhb_comm_df_cleaned, "total_qty_sold","total_kg_sold")
jhb_con_df_cleaned= sold_qty_kg(jhb_con_df_cleaned, "qty_sold","kg_sold")

C:\Users\pitso\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\pitso\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\pitso\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\pitso\Anaconda3\lib

In [35]:
# Looking at the first five rows
#jhb_comm_df_cleaned.head()
jhb_con_df_cleaned.head()

,rowid,date,commodity,container,qty_available,average_price_per_kg,MTD_total_value_sold_(R),total_value_sold_(R),Total_quatity_sold,MTD_Total_quatity_sold,Total_kg_sold,MTD_total_kg_sold
0,1,20 August 2020,AMADUMBE,20KG POCKET,2,R0,39870.0,0.0,0.0,97.0,0.0,1940.0
1,2,20 August 2020,APPLES,10 X 1KG ECONO PACK CARTON,17,R10,41932.0,100.0,1.0,547.0,10.0,5470.0
2,3,20 August 2020,APPLES,11KG JUMBLE CARTON,343,R6.36,218914.0,1190.0,17.0,3170.0,187.0,34870.0
3,4,20 August 2020,APPLES,12 X 1KG ECONO PACK CARTON,3233,R7.86,1454572.0,40738.0,432.0,17353.0,5184.0,208236.0
4,5,20 August 2020,APPLES,12.5KG M6 CARTON,12,R0,16903.0,0.0,0.0,286.0,0.0,3575.0


## The function to change columns to numeric 

In [65]:
def to_numeric(df,column):
    """
    Takes a dataframe and one column that have numbers and a comma as a character.
    Removes the comma and change the column to numeric.
    Returns a dataframe with the column changed to be a numeric column
    """ 
    for i in range(len(df[column])):
        df[column][i]=df[column][i].replace("R","").replace(",","")
    df[column]= pd.to_numeric(df[column])
    return  df

## Changing the columns in the dataframes to be numric

In [16]:
# Cleaning CapeTown Dataframe
cpt_df["date_updated"]=pd.to_datetime(cpt_df["date_updated"])
cpt_df=to_numeric(cpt_df,"mass")
cpt_df=to_numeric(cpt_df, "low_price")
cpt_df=to_numeric(cpt_df,"high_price")
cpt_df=to_numeric(cpt_df,"avg_price")

C:\Users\pitso\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
cpt_df.head()

,item,description,grade,container,container_count,mass,low_price,high_price,avg_price,date_updated
0,MUBR,Mushroom Brown,,Prepack Punnet (5kg),0,5.0,600.0,600.0,600.0,2020-08-25 11:44:00
1,MUOY,Mushroom Oyster,,Prepack Punnet (10kg),0,10.0,1000.0,1000.0,1000.0,2020-08-25 11:44:00
2,MUWH,Mushroom White,,Prepack Punnet (5kg),0,5.0,250.0,250.0,250.0,2020-08-25 11:44:00
3,NEDB,NECTARINES DIAMOND BRIGHT,1M,10*500g Punnet,0,5.0,250.0,250.0,250.0,2020-08-25 11:44:00
4,NEOT,NECTARINES OTHER,2M,10*500g Punnet,0,5.0,220.0,220.0,220.0,2020-08-25 11:44:00


In [9]:
#Cleaning durban dataframe
dbn_df["Date"]=pd.to_datetime(dbn_df["Date"])
dbn_df=to_numeric(dbn_df,"Weight_Kg")
dbn_df=to_numeric(dbn_df,"Low_Price")
dbn_df=to_numeric(dbn_df,"High_Price")
dbn_df=to_numeric(dbn_df,"Average_Price")
dbn_df=to_numeric(dbn_df,"Sales_Total")
dbn_df=to_numeric(dbn_df,"Total_Qty_Sold")
dbn_df=to_numeric(dbn_df,"Total_Kg_Sold")
dbn_df=to_numeric(dbn_df,"Stock_On_Hand")
dbn_df.head()

C:\Users\pitso\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Commodities,Weight_Kg,Size_Grade,Container,Province,Low_Price,High_Price,Average_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,Date
0,POTATO SIFRA (WASHED),10.0,1U,PP100,WESTERN FREESTATE,25.0,40.0,26.35,15785.0,599,5990.0,0,2020-06-24
1,POTATO SIFRA (WASHED),10.0,1Z,PP100,WESTERN FREESTATE,20.0,60.0,50.41,108129.0,2145,21450.0,1369,2020-06-24
2,POTATO SIFRA (WASHED),10.0,1Z,PP100,NORTH WEST,30.0,30.0,30.00,150.0,5,50.0,18,2020-06-24
3,POTATO SIFRA (WASHED),10.0,2M,PP100,WESTERN FREESTATE,30.0,32.0,30.41,3254.0,107,1070.0,3,2020-06-24
4,POTATO SIFRA (WASHED),10.0,3M,PP100,WESTERN FREESTATE,25.0,25.0,25.00,11000.0,440,4400.0,0,2020-06-24


In [67]:
# Cleaning Johannesburg Dataframe
jhb_comb_df["date"]=pd.to_datetime(jhb_comb_df["date"])
jhb_comb_df=to_numeric(jhb_comb_df, "total_value_sold")
jhb_comb_df=to_numeric(jhb_comb_df, "average")
jhb_comb_df=to_numeric(jhb_comb_df, "highest_price")
jhb_comb_df=to_numeric(jhb_comb_df, "ave_per_kg")
jhb_comb_df=to_numeric(jhb_comb_df, "highest_price_per_kg")

C:\Users\pitso\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Split the product combination into several product columns

In [8]:
pd.set_option('display.max_colwidth', None)

In [ ]:
cpt_df_cleaned = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Capetown_Fresh_produce_market_cleaned',cnxn)

In [7]:
cpt_df_cleaned.head()

,index,item,description,grade,container,container_count,mass,low_price,high_price,avg_price,date_updated
0,0,APCR,APPLE CRIPPS RED,1M,ECONOPACK (12kg),150,12.0,110.0,110.0,110.00,2020-08-21 11:36:00
1,1,APER,APPLE EARLY RED ONE,1L,MARK 4 (18.3kg),100,18.3,120.0,120.0,120.00,2020-08-21 11:36:00
2,2,APER,APPLE EARLY RED ONE,1M,MARK 4 (18.3kg),135,18.3,120.0,140.0,121.23,2020-08-21 11:36:00
3,3,APER,APPLE EARLY RED ONE,1X,MARK 4 (18.3kg),90,18.3,120.0,120.0,120.00,2020-08-21 11:36:00
4,4,APGA,APPLE GALA,1L,MARK 4 (18.3kg),110,18.3,160.0,160.0,160.00,2020-08-21 11:36:00


In [14]:
cpt_df_cleaned[['description']].head()

,description
0,APPLE CRIPPS RED
1,APPLE EARLY RED ONE
2,APPLE EARLY RED ONE
3,APPLE EARLY RED ONE
4,APPLE GALA


In [19]:
cpt_df_cleaned[['prod_descrip','prod_type']] = cpt_df_cleaned.description.str.split(' ',1,expand=True)

In [23]:
cpt_df_cleaned.tail()

,index,item,description,grade,container,container_count,mass,low_price,high_price,avg_price,date_updated,prod_descrip,prod_type
4285,4285,APRG,APPLE ROYAL GALA,1U,MARK 4 (18.3kg),216,18.3,50.0,50.0,50.00,2020-01-09 11:48:00,APPLE,ROYAL GALA
4286,4286,APRG,APPLE ROYAL GALA,1X,MARK 4 (18.3kg),90,18.3,160.0,160.0,160.00,2020-01-09 11:48:00,APPLE,ROYAL GALA
4287,4287,APSK,APPLE STARKING,1S,ECONOPACK (12kg),198,12.0,110.0,110.0,110.00,2020-01-09 11:48:00,APPLE,STARKING
4288,4288,APSK,APPLE STARKING,1L,MARK 4 (18.3kg),110,18.3,140.0,170.0,154.69,2020-01-09 11:48:00,APPLE,STARKING
4289,4289,APTR,APPLE TOP RED,1M,BAG/PACKET,0,1.5,11.0,11.0,11.00,2020-01-09 11:48:00,APPLE,TOP RED


In [25]:
cpt_df_cleaned.shape

(4290, 13)

In [59]:
cpt_df_cleaned = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo._Fresh_produce_market_cleaned',cnxn)

In [68]:
jhb_comb_df[['product_combination']].head()

,product_combination
0,"*,*,L,*,*"
1,"GOLDEN DELICIOUS,CL 1,*,*,*"
2,"ROYAL GALA,CL 1,*,*,*"
3,"GOLDEN DELICIOUS,CL 2,L,*,*"
4,"TOPRED,CL 2,L,*,*"


In [69]:
jhb_comb_df[['prod_name','prod_class','prod_size','prod_unknown','prod_colour']] = jhb_comb_df.product_combination.str.split(',',expand=True)

In [70]:
jhb_comb_df.head()

,rowid,date,commodity,container,unit_mass,product_combination,total_value_sold,total_qty_sold,total_kg_sold,average,highest_price,ave_per_kg,highest_price_per_kg,prod_name,prod_class,prod_size,prod_unknown,prod_colour
0,1,2020-08-20,AMADUMBE,20KG POCKET,20,"*,*,L,*,*",0.0,0,0.0,0.0,0.0,0.0,0.0,*,*,L,*,*
1,2,2020-08-20,APPLES,10 X 1KG ECONO PACK CARTON,10,"GOLDEN DELICIOUS,CL 1,*,*,*",100.0,1,10.0,100.0,100.0,10.0,10.0,GOLDEN DELICIOUS,CL 1,*,*,*
2,3,2020-08-20,APPLES,10 X 1KG ECONO PACK CARTON,10,"ROYAL GALA,CL 1,*,*,*",0.0,0,0.0,0.0,0.0,0.0,0.0,ROYAL GALA,CL 1,*,*,*
3,4,2020-08-20,APPLES,11KG JUMBLE CARTON,11,"GOLDEN DELICIOUS,CL 2,L,*,*",0.0,0,0.0,0.0,0.0,0.0,0.0,GOLDEN DELICIOUS,CL 2,L,*,*
4,5,2020-08-20,APPLES,11KG JUMBLE CARTON,11,"TOPRED,CL 2,L,*,*",0.0,0,0.0,0.0,0.0,0.0,0.0,TOPRED,CL 2,L,*,*


In [71]:
jhb_comb_df.shape

(22948, 18)

In [45]:
#jhb_con_df_cleaned=to_numeric(jhb_con_df_cleaned, "average_price_per_kg")

In [23]:
woolworths_df=to_numeric(woolworths_df, "Price")
woolworths_df["Date"]=pd.to_datetime(woolworths_df["Date"])

pnp_df=to_numeric(pnp_df, "Price")
pnp_df["Date"]=pd.to_datetime(pnp_df["Date"])

shoprite_df=to_numeric(shoprite_df, "Price")
shoprite_df["Date"]=pd.to_datetime(shoprite_df["Date"])

In [48]:
woolworths_df.head()

,item,Price,Date
0,Queen Pineapples 2Pk,32.99,2020-08-31
1,Four Leaf Salad 180g,24.99,2020-08-31
2,Raspberries 160g,39.99,2020-08-31
3,Portabellini Mushrooms 250g,27.99,2020-08-31
4,Colour Peppers 3Pk,36.99,2020-08-31


In [50]:
plot(woolworths_df)

In [51]:
plot(pnp_df)

## Creating an Engine to send the cleaned tables to RDS instace 

In [1]:

params = urllib.parse.quote_plus()
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

"""""
@event.listens_for(engine, 'before_cursor_execute')
def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
    if executemany:
        cursor.fast_executemany = True
        cursor.commit()
"""""


Username = 'admin'
Password = 'fXToh9AXt30Yi7dGdlI2'
Host = 'incubate-inventory-management.cnzbp4ndrpos.eu-west-1.rds.amazonaws.com'
Port = '1433'
Database = 'Fresh_Produce_Market_Data'

conn = create_engine(
    'mssql+pymssql://' +
    Username + ':' + Password + '@' + Host + ':' + Port + '/' + Database
)

In [81]:
insp = reflection.Inspector.from_engine(engine)
print(insp.get_table_names())

['Capetown_Fresh_produce_market', 'Capetown_Fresh_produce_market_cleaned', 'container', 'Date', 'Durban_Fresh_produce_market', 'Durban_Fresh_produce_market_cleaned', 'inventory', 'Joburg_Fresh_produce_combined_cleaned', 'Joburg_Fresh_produce_commodity_cleaned', 'Joburg_Fresh_produce_commodity_raw', 'Joburg_Fresh_produce_container_cleaned', 'Joburg_Fresh_produce_container_raw', 'Joburg_Fresh_produce_product_combination_raw', 'Joburg_Fresh_produce_scrapping_date', 'master_date', 'PickNPay_Prices', 'PickNPay_Prices_cleaned', 'product', 'product_combination', 'RetailerPrices', 'RetailerProducts', 'Retailers', 'sales', 'Shoprite_Prices', 'Shoprite_Prices_cleaned', 'sysdiagrams', 'woolworths_Prices', 'Woolworths_Prices_cleaned']


## Adding the tables in the RDS instance

In [31]:
jhb_comm_df_cleaned.to_sql('Joburg_Fresh_produce_commodity_cleaned', con=engine)

In [30]:
jhb_con_df_cleaned.to_sql('Joburg_Fresh_produce_container_cleaned', con=engine)

In [74]:
sql = 'DROP TABLE IF EXISTS Joburg_Fresh_produce_combined_cleaned;'
engine.execute(sql)
cur.commit()
jhb_comb_df.to_sql('Joburg_Fresh_produce_combined_cleaned', con=engine,if_exists="append",chunksize=2000)

In [50]:
sql = 'DROP TABLE IF EXISTS Capetown_Fresh_produce_market_cleaned;'
engine.execute(sql)
cur.commit()
cpt_df_cleaned.to_sql('Capetown_Fresh_produce_market_cleaned', con=engine,if_exists="append",chunksize=500)

In [26]:
dbn_df.to_sql('Durban_Fresh_produce_market_cleaned', con=engine)


In [27]:
woolworths_df.to_sql('Woolworths_Prices_cleaned', con=engine)


In [28]:
pnp_df.to_sql('PickNPay_Prices_cleaned', con=engine)


In [29]:
shoprite_df.to_sql('Shoprite_Prices_cleaned', con=engine)

## Interacting with the database in RDS to query the new created tables

In [33]:
engine.execute("SELECT * FROM Joburg_Fresh_produce_commodity_cleaned").fetchall()

[(0, 1, '20 August 2020', 'AMADUMBE', 2, 39870.0, 0.0, 0.0, 97.0, 0.0, 1940.0),
 (1, 2, '20 August 2020', 'APPLES', 91755, 22664221.0, 1205932.0, 13799.0, 261296.0, 157462.0, 3163863.0),
 (2, 3, '20 August 2020', 'ARTICHOKES', 1, 53100.0, 600.0, 4.0, 439.0, 3.0, 522.0),
 (3, 4, '20 August 2020', 'ASPARAGUS', 8, 258975.0, 34000.0, 50.0, 359.0, 250.0, 1795.0),
 (4, 5, '20 August 2020', 'ATCHARA', 207, 1351.2, 0.0, 0.0, 23.0, 0.0, 65.0),
 (5, 6, '20 August 2020', 'AVOCADOS', 14429, 11358074.4, 1058716.0, 9569.0, 124016.0, 54059.0, 747703.0),
 (6, 7, '20 August 2020', 'BABY BEET', 7, 1462.0, 0.0, 0.0, 31.0, 0.0, 63.0),
 (7, 8, '20 August 2020', 'BABY BRINJALS', 0, 1920.0, 1320.0, 9.0, 13.0, 27.0, 39.0),
 (8, 9, '20 August 2020', 'BABY BUTTERNUT', 6, 117371.0, 21752.0, 86.0, 425.0, 403.0, 1925.0),
 (9, 10, '20 August 2020', 'BABY CABBAGE', 197, 254455.0, 21824.0, 196.0, 2592.0, 783.0, 9422.0),
 (10, 11, '20 August 2020', 'BABY CARROTS', 301, 163250.0, 16210.0, 84.0, 946.0, 484.0, 4907.0),
 

In [34]:
engine.execute('SELECT * from Joburg_Fresh_produce_container_cleaned').fetchall()

[(0, 1, '20 August 2020', 'AMADUMBE', '20KG POCKET', 2, 0.0, 39870.0, 0.0, 0.0, 97.0, 0.0, 1940.0),
 (1, 2, '20 August 2020', 'APPLES', '10 X 1KG ECONO PACK CARTON', 17, 10.0, 41932.0, 100.0, 1.0, 547.0, 10.0, 5470.0),
 (2, 3, '20 August 2020', 'APPLES', '11KG JUMBLE CARTON', 343, 6.36, 218914.0, 1190.0, 17.0, 3170.0, 187.0, 34870.0),
 (3, 4, '20 August 2020', 'APPLES', '12 X 1KG ECONO PACK CARTON', 3233, 7.86, 1454572.0, 40738.0, 432.0, 17353.0, 5184.0, 208236.0),
 (4, 5, '20 August 2020', 'APPLES', '12.5KG M6 CARTON', 12, 0.0, 16903.0, 0.0, 0.0, 286.0, 0.0, 3575.0),
 (5, 6, '20 August 2020', 'APPLES', '12.5KG M6 CONTAINER', 0, 8.8, 7700.0, 7700.0, 70.0, 70.0, 875.0, 875.0),
 (6, 7, '20 August 2020', 'APPLES', '18.50KG CARTON', 34346, 8.25, 7339878.0, 411595.0, 2696.0, 53757.0, 49876.0, 994191.0),
 (7, 8, '20 August 2020', 'APPLES', '3 KG ECONO PACK BOX', 211, 0.0, 59208.0, 0.0, 0.0, 1490.0, 0.0, 6675.0),
 (8, 9, '20 August 2020', 'APPLES', '300KG BULK BIN', 129, 5.17, 1278425.0, 7755

In [80]:
engine.execute('SELECT * from Joburg_Fresh_produce_combined_cleaned').fetchall()

[(0, 1, datetime.datetime(2020, 8, 20, 0, 0), 'AMADUMBE', '20KG POCKET', 20, '*,*,L,*,*', 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, '*', '*', 'L', '*', '*'),
 (1, 2, datetime.datetime(2020, 8, 20, 0, 0), 'APPLES', '10 X 1KG ECONO PACK CARTON', 10, 'GOLDEN DELICIOUS,CL 1,*,*,*', 100.0, 1, 10.0, 100.0, 100.0, 10.0, 10.0, 'GOLDEN DELICIOUS', 'CL 1', '*', '*', '*'),
 (2, 3, datetime.datetime(2020, 8, 20, 0, 0), 'APPLES', '10 X 1KG ECONO PACK CARTON', 10, 'ROYAL GALA,CL 1,*,*,*', 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 'ROYAL GALA', 'CL 1', '*', '*', '*'),
 (3, 4, datetime.datetime(2020, 8, 20, 0, 0), 'APPLES', '11KG JUMBLE CARTON', 11, 'GOLDEN DELICIOUS,CL 2,L,*,*', 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 'GOLDEN DELICIOUS', 'CL 2', 'L', '*', '*'),
 (4, 5, datetime.datetime(2020, 8, 20, 0, 0), 'APPLES', '11KG JUMBLE CARTON', 11, 'TOPRED,CL 2,L,*,*', 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 'TOPRED', 'CL 2', 'L', '*', '*'),
 (5, 6, datetime.datetime(2020, 8, 20, 0, 0), 'APPLES', '11KG JUMBLE CARTON', 11, 'TOPRED,CL 2,M,*

In [73]:
engine.execute('SELECT * from Capetown_Fresh_produce_market_cleaned').fetchall()

[(0, 0, 'APCR', 'APPLE CRIPPS RED', '1M', 'ECONOPACK (12kg)', 150, 12.0, 110.0, 110.0, 110.0, datetime.datetime(2020, 8, 21, 11, 36), 'APPLE', 'CRIPPS RED'),
 (1, 1, 'APER', 'APPLE EARLY RED ONE', '1L', 'MARK 4 (18.3kg)', 100, 18.3, 120.0, 120.0, 120.0, datetime.datetime(2020, 8, 21, 11, 36), 'APPLE', 'EARLY RED ONE'),
 (2, 2, 'APER', 'APPLE EARLY RED ONE', '1M', 'MARK 4 (18.3kg)', 135, 18.3, 120.0, 140.0, 121.23, datetime.datetime(2020, 8, 21, 11, 36), 'APPLE', 'EARLY RED ONE'),
 (3, 3, 'APER', 'APPLE EARLY RED ONE', '1X', 'MARK 4 (18.3kg)', 90, 18.3, 120.0, 120.0, 120.0, datetime.datetime(2020, 8, 21, 11, 36), 'APPLE', 'EARLY RED ONE'),
 (4, 4, 'APGA', 'APPLE GALA', '1L', 'MARK 4 (18.3kg)', 110, 18.3, 160.0, 160.0, 160.0, datetime.datetime(2020, 8, 21, 11, 36), 'APPLE', 'GALA'),
 (5, 5, 'APGD', 'APPLE GOLDEN DELICIOUS', '1M', 'PACKET', 0, 1.5, 8.0, 8.0, 8.0, datetime.datetime(2020, 8, 21, 11, 36), 'APPLE', 'GOLDEN DELICIOUS'),
 (6, 6, 'APGD', 'APPLE GOLDEN DELICIOUS', '1M', 'BAG/PACK

In [37]:
engine.execute('SELECT * from Durban_Fresh_produce_market_cleaned').fetchall()

[]

In [38]:
engine.execute('SELECT * from Woolworths_Prices_cleaned').fetchall()

[(0, 'Queen Pineapples 2Pk', 32.99, datetime.datetime(2020, 8, 31, 0, 0)),
 (1, 'Four Leaf Salad 180g', 24.99, datetime.datetime(2020, 8, 31, 0, 0)),
 (2, 'Raspberries 160g', 39.99, datetime.datetime(2020, 8, 31, 0, 0)),
 (3, 'Portabellini Mushrooms 250g', 27.99, datetime.datetime(2020, 8, 31, 0, 0)),
 (4, 'Colour Peppers 3Pk', 36.99, datetime.datetime(2020, 8, 31, 0, 0)),
 (5, 'Baby Potatoes 700g', 17.99, datetime.datetime(2020, 8, 31, 0, 0)),
 (6, 'Everyday Large Potatoes 2Kg', 34.99, datetime.datetime(2020, 8, 31, 0, 0)),
 (7, 'Rosa Tomatoes 200g', 15.99, datetime.datetime(2020, 8, 31, 0, 0)),
 (8, 'Granny Smith Apples 1.5Kg', 28.99, datetime.datetime(2020, 8, 31, 0, 0)),
 (9, 'Mixed Mushrooms 400g', 29.99, datetime.datetime(2020, 8, 31, 0, 0)),
 (10, 'Blueberries 400g', 69.99, datetime.datetime(2020, 8, 31, 0, 0)),
 (11, 'Ripen at Home Avocados 1Kg', 39.99, datetime.datetime(2020, 8, 31, 0, 0)),
 (12, 'Bulk Roasting Vegetables 800g', 39.99, datetime.datetime(2020, 8, 31, 0, 0)),
 (

In [39]:
engine.execute('SELECT * from PickNPay_Prices_cleaned').fetchall()

[(0, 'PnP Oranges 4s', 24.99, datetime.datetime(2020, 8, 24, 0, 0)),
 (1, 'PnP Plums 750g', 34.99, datetime.datetime(2020, 8, 24, 0, 0)),
 (2, 'PnP Starking Apples 1.5kg', 29.99, datetime.datetime(2020, 8, 24, 0, 0)),
 (3, 'PnP Baby Papaya 2s', 29.99, datetime.datetime(2020, 8, 24, 0, 0)),
 (4, 'PnP Kids Yellow Apples 1kg', 17.99, datetime.datetime(2020, 8, 24, 0, 0)),
 (5, 'PnP Lemons 850g', 26.99, datetime.datetime(2020, 8, 24, 0, 0)),
 (6, 'PnP Juicing Oranges 2kg', 24.99, datetime.datetime(2020, 8, 24, 0, 0)),
 (7, 'PnP Lemons 4s', 27.99, datetime.datetime(2020, 8, 24, 0, 0)),
 (8, 'PnP Packham Pears 1.5kg', 26.99, datetime.datetime(2020, 8, 24, 0, 0)),
 (9, 'PnP Grapefruit Carry Bag 2.5kg', 34.99, datetime.datetime(2020, 8, 24, 0, 0)),
 (10, 'PnP Grapefruit 4s', 34.99, datetime.datetime(2020, 8, 24, 0, 0)),
 (11, 'PnP Oranges Bag 2.5kg', 26.99, datetime.datetime(2020, 8, 24, 0, 0)),
 (12, 'PnP Limes 350g', 29.99, datetime.datetime(2020, 8, 24, 0, 0)),
 (13, 'PnP Golden Delicious A

In [40]:
engine.execute('SELECT * from Shoprite_Prices_cleaned').fetchall()

[(0, 'Grapefruit In Bag 2kg', 24.99, datetime.datetime(2020, 8, 25, 0, 0)),
 (1, 'Safari Pitted Dates Pack 250g', 16.99, datetime.datetime(2020, 8, 25, 0, 0)),
 (2, 'Lemon Per kg', 18.99, datetime.datetime(2020, 8, 25, 0, 0)),
 (3, 'Naartjie Per kg', 19.99, datetime.datetime(2020, 8, 25, 0, 0)),
 (4, 'Grapefruit Per kg', 14.99, datetime.datetime(2020, 8, 25, 0, 0)),
 (5, 'Orange Per kg', 14.99, datetime.datetime(2020, 8, 25, 0, 0)),
 (6, 'Packham Pears Pack 1.5kg', 19.99, datetime.datetime(2020, 8, 25, 0, 0)),
 (7, 'Gala Royal Apples Bag 1.5kg', 26.99, datetime.datetime(2020, 8, 25, 0, 0)),
 (8, 'Forelle Pears 1.5kg', 23.99, datetime.datetime(2020, 8, 25, 0, 0)),
 (9, 'Delicious Golden Apples Bag 1.5kg', 18.99, datetime.datetime(2020, 8, 25, 0, 0)),
 (10, 'Red Top Apples 1.5kg', 26.99, datetime.datetime(2020, 8, 25, 0, 0)),
 (11, 'Granny Smith Apples 1.5kg', 16.99, datetime.datetime(2020, 8, 25, 0, 0)),
 (12, 'Papaya Loose Per kg', 16.99, datetime.datetime(2020, 8, 25, 0, 0)),
 (13, 'P

## Dropping a table

**We can drop any table if we want to recreate it, using the below code**

In [11]:
sql = 'Drop TABLE Durban_Fresh_produce_market_cleaned;'
engine.execute(sql)
cur.commit()

In [42]:
#sql = 'Drop TABLE Joburg_Fresh_produce_combined_cleaned;'
#engine.execute(sql)

In [47]:
#from sqlalchemy.engine import reflection,inspect
